# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


In [11]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay
from aux_func import evaluate_model, cargar_modelo
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2021-12-18 18:46:33 +01:00)


In [2]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

time: 1.88 s (started: 2021-12-18 18:42:34 +01:00)


In [3]:
#Cargamos pipeline preprocesado
preprocessor = cargar_modelo('../models/preprocessor.pickle')

time: 172 ms (started: 2021-12-18 18:42:36 +01:00)


# XGBoost

El XGBoost o extreme gradient boosting es un procedimiento para el ensamblado de algoritmos de Machine Learning. Sus siglas hacen referencia a eXtreme Gradient Boosting. Este método, al contrario que el bagging, no procesa en paralelo los modelos, es más complejo.

El procedimiento es secuencial. Se empieza con un modelo, que entrena con unos datos, y se evalua con un conjunto de test. Una vez evaluado el desempeño de ese primer modelo, se localizan aquellas observaciones en las que el modelo ha tenido más problemas y ha fallado. Estas observaciones tienen más probabilidad de aparecer en los datos para el siguiente modelo, y así sucesivamente. Lo que busca es pulir una y otra vez las observaciones que son difíciles de clasificar.

Es un método potente y complejo, flexible en el sentido de que los modelos no tienen por qué ser del mismo tipo.

Procedemos a entrenar un modelo de XGBoost con nuestros datos y parámetros por defecto.

![Highway](https://programmerclick.com/images/517/897f66771ca1b47c9ed91516b58a621d.png)

In [4]:
clf = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('clasificador', XGBClassifier(n_jobs=-1, random_state=0))])

time: 0 ns (started: 2021-12-18 18:42:42 +01:00)


In [5]:
clf.fit(xtrain, ytrain)

[18:43:15] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 3min 2s (started: 2021-12-18 18:42:46 +01:00)


In [6]:
with open('../models/XGBoost.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 31 ms (started: 2021-12-18 18:45:49 +01:00)


In [7]:
with open('../models/XGBoost.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 16 ms (started: 2021-12-18 18:45:49 +01:00)


Generamos las predicciones sobre los datos de validación y evaluamos el modelo.

In [8]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8541541641828294
Accuracy of the model: 0.9848714106843199

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    799946
           1       0.50      0.01      0.03     12291

    accuracy                           0.98    812237
   macro avg       0.74      0.51      0.51    812237
weighted avg       0.98      0.98      0.98    812237


Confusion matrix: 
[[799774    172]
 [ 12116    175]]

time: 8.31 s (started: 2021-12-18 18:45:49 +01:00)


## Ajuste del umbral de predicción

De la misma manera que ocurría en el modelo Random Forest, nuestro modelo obtiene un valor de recall muy bajo para la clase minoritaria.
Procedemos a ajustar el umbral de predicción siguiendo el punto óptimo según la curva ROC.

In [9]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.014962, G-Mean=0.770
ROC-AUC score of the model: 0.8541541641828294
Accuracy of the model: 0.775515028249144

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.78      0.87    799946
           1       0.05      0.77      0.09     12291

    accuracy                           0.78    812237
   macro avg       0.52      0.77      0.48    812237
weighted avg       0.98      0.78      0.86    812237


Confusion matrix: 
[[620499 179447]
 [  2888   9403]]

time: 1.58 s (started: 2021-12-18 18:45:57 +01:00)


Podemos observar como el ajuste del threshold dota al modelo de un mayor recall para los casos de la clase minoritaria, lo cual nos interesa desde un punto de vista práctico a pesar de reducir la precisión y accuracy del modelo.

## Comprobación de overfitting

Comprobamos si el modelo sufre de overfitting, realizando una predicción sobre la serie de entrenamiento.

In [10]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.015049, G-Mean=0.782
ROC-AUC score of the model: 0.8666278527727284
Accuracy of the model: 0.7778137210694549

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.78      0.87   3200049
           1       0.05      0.79      0.10     48896

    accuracy                           0.78   3248945
   macro avg       0.52      0.78      0.48   3248945
weighted avg       0.98      0.78      0.86   3248945


Confusion matrix: 
[[2488656  711393]
 [  10478   38418]]

time: 34.2 s (started: 2021-12-18 18:45:59 +01:00)


A diferencia del modelo Random Forest, XGBoost no ha realizado un ajuste tan pronunciado sobre los datos de entrenamiento.
Esto puede deberse a la naturaleza y proceso interno de los algoritmos, ya que el modelo Random Forest gana robustez cuanto más profundo y optimizado está.
Sin embargo, el modelo XGBoost tiende al overfitting cuando aumentamos el poder de computación.

# XGBoost con SMOTE

Prueba de aplicación del mismo modelo a la versión de los datos que incluye Oversampling de la clase minoritaria mediante SMOTE.

In [14]:
from imblearn.pipeline import Pipeline
clf = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('smote', SMOTE(sampling_strategy=0.4, n_jobs=-1)),
    ('clasificador', XGBClassifier(n_jobs=-1, random_state=0))])

time: 0 ns (started: 2021-12-18 18:48:14 +01:00)


In [15]:
clf.fit(xtrain, ytrain)

[18:49:36] WARNING: D:\bld\xgboost-split_1637426510059\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 7min 14s (started: 2021-12-18 18:48:19 +01:00)


In [16]:
with open('../models/XGBoost_smote.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 63 ms (started: 2021-12-18 18:55:33 +01:00)


In [17]:
with open('../models/XGBoost_smote.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 31 ms (started: 2021-12-18 18:55:33 +01:00)


In [18]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8335443595477434
Accuracy of the model: 0.9847039718702792

Classification report: 
              precision    recall  f1-score   support

           0       0.99      1.00      0.99    799946
           1       0.35      0.01      0.02     12291

    accuracy                           0.98    812237
   macro avg       0.67      0.51      0.51    812237
weighted avg       0.98      0.98      0.98    812237


Confusion matrix: 
[[799656    290]
 [ 12134    157]]

time: 8.98 s (started: 2021-12-18 18:55:33 +01:00)


El modelo sin aplicar el ajuste de threshold no es lo suficientemente descriptivo como para poder compararlo con su versión análoga. Procedemos a realizar dicho ajuste.

## Ajuste del umbral de predicción

In [19]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.022001, G-Mean=0.750
ROC-AUC score of the model: 0.8335443595477434
Accuracy of the model: 0.74873589851238

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.75      0.85    799946
           1       0.04      0.75      0.08     12291

    accuracy                           0.75    812237
   macro avg       0.52      0.75      0.47    812237
weighted avg       0.98      0.75      0.84    812237


Confusion matrix: 
[[598906 201040]
 [  3046   9245]]

time: 1.66 s (started: 2021-12-18 18:55:42 +01:00)


Tras realizar el ajuste del modelo, podemos afirmar que en nuestro caso concreto y para el tratamiento de los datos que hemos realizado, no existe una razón por la que usar la versión Oversample de los datos.
Los resultados obtenidos son peores respecto del modelo generado con datos sin alterar.